In [260]:
import numpy as np
import onnx
import onnxruntime as ort
import time
import matplotlib.pyplot as plt
import csv

class Multiply_experiment_class():
    #TODO: Set the input size to the size of a layer in the TINYYOLO 
    def __init__(self):
        self.in1 = np.random.random([4,4]).astype(np.float32)
        self.in2 = np.random.random([4,4]).astype(np.float32)
        self.ort_runtime=[]
        self.out1 = np.random.random([4,4]).astype(np.float32)
        
    def expected_res_mul(self):
        res = np.multiply(self.in1,self.in2)
        return res

    def test_ort_mul(self, model_name):
        model_load = onnx.load(model_name)
        expected_result = self.expected_res_mul()
        input_dict = {"in1": self.in1, "in2": self.in2}
        sess = ort.InferenceSession(model_load.SerializeToString())
        self.out1 = sess.run(None,input_dict)
        return np.allclose(self.out1,expected_result)
           
    
    def throughput_test(self,interations, filename = "ort_results.csv"):
        for i in range(interations):
            start = time.time()
            self.test_ort_mul("multiply_model.onnx")
            end = time.time()
            self.ort_runtime.append(end-start)       
        self.results_publisher()
        np.savetxt(filename , np.asarray(ort_runtime), delimiter=',')
        return self.ort_runtime
    
    def results_publisher(self):
        print("Interations = %d" %(len(self.ort_runtime)))
        print("Min = %f ms" %(np.mean(self.ort_runtime)*1000))
        print("Max = %f ms" %(np.max(self.ort_runtime)*1000))
        print("Mean = %f ms" %(np.min(self.ort_runtime)*1000))

In [261]:
mul_exp = Multiply_experiment_class()    

In [262]:
ort_rt = mul_exp.throughput_test(100)

Interations = 100
Min = 2.848241 ms
Max = 11.878967 ms
Mean = 0.996828 ms


In [ ]:
Mul1_node = onnx.helper.make_node(
    'Mul',
    inputs=['in1', 'in2'],
    outputs=['out1'],
    name='Mul1'
)

In [ ]:
in1 = onnx.helper.make_tensor_value_info("in1", onnx.TensorProto.FLOAT, [4, 4])
in2 = onnx.helper.make_tensor_value_info("in2", onnx.TensorProto.FLOAT, [4, 4])
out1 = onnx.helper.make_tensor_value_info("out1", onnx.TensorProto.FLOAT, [4, 4])

In [ ]:
 graph = onnx.helper.make_graph(
        nodes=[
            Mul1_node,
        ],
        name="simple_graph",
        inputs=[in1, in2],
        outputs=[out1],
        value_info=[
        ],
    )

In [ ]:
onnx_model = onnx.helper.make_model(graph, producer_name="multiply-model")
onnx.save(onnx_model, 'multiply_model.onnx')